In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
driver = webdriver.Chrome('/home/sara/ChromeDriver/chromedriver')

In [4]:
urls_dict = {"São Paulo": "https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/"}

errors = []
results = []

for bairro, url in urls_dict.items():
    current_url = url
    driver.get(url)
    sleep(2)
    actions = ActionChains(driver)

    try:
        driver.find_element_by_class_name("cookie-notifier__cta").click()
    except:
            print("No cookies!")
    
    for i in tqdm(range(5), desc=bairro):
        
        
        sleep(5)
        main_div = driver.find_element_by_class_name("results-main__panel")
        properties = main_div.find_elements_by_class_name("js-property-card")
        paginator = driver.find_element_by_class_name("js-results-pagination")
        next_page = paginator.find_element_by_xpath("//a[@title='Próxima página']") #/html/body/main/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a
        
        for i,apartment in enumerate(properties):
            url = apartment.find_element_by_class_name("js-card-title").get_attribute("href")
            apto_id = url.split("id-")[-1][:-1]
            header = apartment.find_element_by_class_name("property-card__title").text
            address = apartment.find_element_by_class_name("property-card__address").text
            area = apartment.find_element_by_class_name("js-property-card-detail-area").text
            rooms = apartment.find_element_by_class_name("js-property-detail-rooms").text
            bathrooms = apartment.find_element_by_class_name("js-property-detail-bathroom").text
            garages = apartment.find_element_by_class_name("js-property-detail-garages").text
            try:
                amenities = apartment.find_element_by_class_name("property-card__amenities").text
            except:
                amenities = None
                errors.append(url)
            price = apartment.find_element_by_class_name("js-property-card-prices").text
            try:
                condo = apartment.find_element_by_class_name("js-condo-price").text
            except:
                condo = None
                errors.append(url)
            crawler = bairro
            crawled_at = datetime.now().strftime("%Y-%m-%d %H:%M")
            results.append({"id": apto_id,
                            "url": url,
                            "header": header,
                            "address": address,
                            "area": area,
                            "rooms": rooms,
                            "bathrooms": bathrooms,
                            "garages": garages,
                            "amenities": amenities,
                            "price": price,
                            "condo": condo,
                            "crawler": crawler,
                            "crawled_at": crawled_at})
        next_page.click()
pd.DataFrame(results).to_csv("full_results.csv", index=False)
driver.close()

São Paulo: 100%|██████████| 5/5 [01:08<00:00, 13.77s/it]


In [5]:
df = pd.DataFrame(results)

In [6]:
df.shape

(180, 13)

In [7]:
df.head()

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
0,2518566682,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos para Aluguel, 126m²","Rua Peixoto Gomide, 601 - Jardim Paulista, São...",126,3 Quartos,2 Banheiros,1 Vaga,Condomínio fechado\nAceita animais\nPortaria 2...,R$ 2.900 /mês,R$ 1.640,São Paulo,2021-05-25 16:57
1,2519455739,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos para Aluguel, 120m²","Avenida Arruda Botelho, 570 - Alto de Pinheiro...",120,3 Quartos,2 Banheiros,3 Vagas,Academia\nElevador\nPortaria 24h\nLavanderia,R$ 4.000 /mês,R$ 1.502,São Paulo,2021-05-25 16:57
2,2518366098,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 4 Quartos para Aluguel, 266m²","Rua dos Caetés, 100 - Perdizes, São Paulo - SP",266,4 Quartos,6 Banheiros,5 Vagas,Mobiliado\nPiscina\nVaranda gourmet\nAcademia\...,R$ 25.000 /mês,R$ 2.000,São Paulo,2021-05-25 16:57
3,2520629099,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 84m²","Rua José Antônio Coelho, 473 - Vila Mariana, S...",84,2 Quartos,3 Banheiros,2 Vagas,Piscina\nVaranda gourmet\nAcademia\nChurrasque...,R$ 4.700 /mês,R$ 1.100,São Paulo,2021-05-25 16:57
4,2518714218,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos para Aluguel, 110m²","Rua Bagé, 230 - Vila Mariana, São Paulo - SP",110,3 Quartos,2 Banheiros,1 Vaga,Piscina\nAcademia\nElevador\nCondomínio fechad...,R$ 4.400 /mês,R$ 1.100,São Paulo,2021-05-25 16:57


In [8]:
df.describe()

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
count,180,180,180,180,180,180,180,180,175,180,165,180,180
unique,180,180,106,175,88,4,6,5,107,91,131,1,2
top,2519037360,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com Quarto para Aluguel, 45m²","Alameda Jaú, 409 - Jardim Paulista, São Paulo ...",50,1 Quarto,1 Banheiro,1 Vaga,Mobiliado\nPiscina\nAcademia\nAr-condicionado\...,R$ 2.200 /mês,R$ 550,São Paulo,2021-05-25 16:57
freq,1,1,7,2,11,75,91,100,13,9,4,180,144
